# Decision Tree

Recently, many kaggle competition winners have used ensemble methods like Random Forest, Gradient Boosting, XGBoost, LightGBM, CatBoost, etc. These methods are very powerful and can achieve very high accuracy, especially when you have lots of **category features**. 

To understand how these ensemble methods work, we need to understand the basic building block of these methods, which is the Decision Tree.



In [8]:
import math
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

A tree has nodes and edges. The nodes are the decision points, and the edges are the outcomes of the decision. The tree starts with a root node and ends with leaf nodes. The root node is the first decision point, and the leaf nodes are the final outcomes.

We can build a decision tree recursively.

In [2]:
class DecisionTree:
    
    def __init__(self, max_depth: int):
        assert max_depth >= 0  # max_depth must be a positive integer
        self.max_depth = max_depth
        if max_depth > 0:
            self.left = DecisionTree(max_depth - 1)
            self.right = DecisionTree(max_depth - 1)

In [3]:
t = DecisionTree(2)

In [4]:
t.max_depth, t.left.max_depth, t.right.max_depth

(2, 1, 1)

Let's say you want to get an apple out of a basket of fruits. You are actually doing classification:

- check the shape of the fruit
    - if it is round, check the color
        - if it is red, it is an apple
        - if it is not red, it is not an apple
    - if it is not round, it is not an apple

In reality, our mind works like a decision tree in nanoseconds.

In machine learning, we usually have a dataset with many features and a target variable. We can build a decision tree to predict the target variable based on the features.

Suppose we have the following table:

| feature_value| color|
|--------------|------|
| 20       | red  |
| 30        | red  |
| 40       | red |
| 50        | green|
| 60        | green|
| 70        | green|

We can build a decision tree to predict the color based on the feature value. The above example gives the extreme scenario where the feature value is strongly associated with the color. Meaning, whenever the feature value is less than 50, the color is red, and whenever the feature value is greater than 50, the color is green.

Our goal is to come up a machenism to build a decision tree based on the data, so that we can predict the target variable based on the features.

In [31]:
class DecisionTree:
    
    def __init__(self, X: pd.DataFrame, y: pd.Series,
                 min_samples_leaf: int = 5,
                 max_depth: int = 6,
                 idxs: np.ndarray = None):
        assert max_depth >= 0
        assert min_samples_leaf > 0
        self.min_samples_leaf, self.max_depth = min_samples_leaf, max_depth
        if isinstance(y, pd.Series):
            y = y.values
        if idxs is None:
            idxs = np.arange(len(y))
        self.X, self.y, self.idxs = X, y, idxs
        self.n, self.m = len(idxs), X.shape[1]
        # value of the node is the mean of y
        self.value = np.mean(y[idxs])
        self.best_score_so_far = float('inf') # initial loss before split finding
        
        if self.max_depth > 0:
            self._maybe_insert_child_nodes()
    
    def _maybe_insert_child_nodes(self):
        pass

In [5]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(as_frame=True, return_X_y=True)

In [6]:
type(X), type(y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [25]:
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [26]:
y.head()

0    151.0
1     75.0
2    141.0
3    206.0
4    135.0
Name: target, dtype: float64

In [14]:
t = DecisionTree(X, y, min_samples_leaf=5, max_depth=5)

## Inserting Child Nodes

When we are searching for an apple, every time we make a decision, we are inserting a child node. For instance, when we check the shape of the fruit, we are inserting a child node. When we check the color of the fruit, we are inserting another child node.

Therefore, we can build a decision tree by inserting child nodes based on the feature values. Read the following code first and then I will explain it.

In [32]:
def _maybe_insert_child_nodes(self):
        for j in range(self.m):
            # there exist a better split for feature j
            # such as value < 50 for color red in our example
            # we will implement this method later
            self._find_better_split(j)
        if self.is_leaf:
            # no need to create child nodes
            return
        x = self.X.values[self.idxs, self.split_feature_idx]
        # get the indexes of the samples that are less than 
        # or equal to the split value
        # this step is to sort the samples to the left and right nodes
        left_idx = np.nonzero(x <= self.threshold)[0]
        right_idx = np.nonzero(x > self.threshold)[0]
        self.left = DecisionTree(self.X, self.y, self.min_samples_leaf, self.max_depth - 1, self.idxs[left_idx])
        self.right = DecisionTree(self.X, self.y, self.min_samples_leaf, self.max_depth - 1, self.idxs[right_idx])
        
def _find_better_split(self, feature_idx):
    pass

@property
def is_leaf(self):
    return self.best_score_so_far == float('inf')

In [18]:
np.nonzero([1, 2] < [4, -2])

(array([0]),)

In [35]:
DecisionTree._maybe_insert_child_nodes = _maybe_insert_child_nodes
DecisionTree._find_better_split = _find_better_split
DecisionTree.is_leaf = is_leaf
t = DecisionTree(X, y, min_samples_leaf=5, max_depth=6)

## Split Finding

Now, let's see the following table with two features and a target variable.

| feature1 | feature2 | target |
|----------|----------|--------|
| 20       | A       | red    |
| 30       | B      | red    |
| 40       | A      | red    |
| 50       | A       | green  |
| 60       | B     | green  |
| 70       | B     | green  |

When any target value is determined by the will of the GOD, there might be some relationship between the features and the target variable. Suppose we have a set of features $(f_1, f_2, f_3, ..., f_n)$ and a target variable $y$.

Those features could be equally important, or some of them could be more important than others, or some combination of features could be more important than others. For instance, we can have $(f_1, f_5, f_n)$ as the most important features. And this combination then will then work together with other features to determine the target variable.

To determine whether the combination of features is important or not, we need a metric to measure the split.

For classification, we will use `majority voting` to determine the target variable. For instance, if we have 3 reds and 2 greens, the target variable will be red. If we have 2 reds and 3 greens, the target variable will be green.

For the regression problem, we will use the `mean` of the target variable. This means we simply take the average of the target variable as the best guess.

Suppose we will train a regression tree:

$$
L = \sum_{i=1}^{n} (y_i - \hat{y_i})^2
$$

where $y_i$ is the actual target value and $\hat{y_i}$ is the predicted target value. For a given node, we can replace $\hat{y_i}$ with the mean of the target values in that node.

$$
L = \sum_{i=1}^{n} (y_i - \bar{y})^2
$$

where $\bar{y}$ is the mean of the target values in that node.

Now, we will expand the above equation:

$$
\begin{aligned}
L & = \sum_{i=1}^{n} (y_i^2 - 2y_i\bar{y} + \bar{y}^2) \\
& = \sum_{i=1}^{n} y_i^2 - 2\bar{y}\sum_{i=1}^{n} y_i + n\bar{y}^2 \\
& = \sum_{i=1}^{n} y_i^2 - 2n\bar{y}^2 + n\bar{y}^2 \\
& = \sum_{i=1}^{n} y_i^2 - n\bar{y}^2 \\
& = \sum_{i=1}^{n} y_i^2 - n\left(\frac{\sum_{i=1}^{n} y_i}{n}\right)^2 \\
& = \sum_{i=1}^{n} y_i^2 - \frac{1}{n} \left (\sum_{i=1}^{n} y_i \right)^2 \\
\end{aligned}
$$

We can then evaluate potential splits by comparing the loss after splitting to the loss before splitting, where the split with the greatest loss reduction is best. Let’s work out a simple expression for the loss reduction from a given split.

There is a strong connection between squared error loss and variance. The variance of a set of values is defined as:

$$
\text{Var}(X) = \frac{1}{n} \sum_{i=1}^{n} (x_i - \bar{x})^2
$$

where $\bar{x}$ is the mean of the values. We can rewrite the variance as:

$$
\text{Var}(X) = \frac{1}{n} \sum_{i=1}^{n} x_i^2 - \bar{x}^2 = \frac{1}{n} \sum_{i=1}^{n} x_i^2 - \frac{1}{n} \left(\sum_{i=1}^{n} x_i \right)^2
$$

Therefore, when we split the data into two groups, the total variance is the sum of the variances of the two groups. The variance reduction is the difference between the total variance and the sum of the variances of the two groups. We will only split the data when the variance reduction is positive.

## Formula for Variance Reduction

Let $I$ be the set of $n$ data instance in the current node, and let $I_L$ and $I_R$ be the set of data instances in the left and right child nodes after the split.

The loss before the split is:

$$
L_{\text{before}} = \sum_{i \in I} y_i^2 - \frac{1}{n} \left(\sum_{i \in I} y_i \right)^2
$$

The loss after the split is:

$$
L_{\text{after}} = I_L + I_R =  \sum_{i \in I_L} y_i^2 - \frac{1}{n_L} \left(\sum_{i \in I_L} y_i \right)^2 + \sum_{i \in I_R} y_i^2 - \frac{1}{n_R} \left(\sum_{i \in I_R} y_i \right)^2
$$

The reduction in loss is:

$$
\begin{aligned}
\text{Reduction} & = L_{\text{after}} - L_{\text{before}} \\
& = \sum_{i \in I_L} y_i^2 - \frac{1}{n_L} \left(\sum_{i \in I_L} y_i \right)^2 + \sum_{i \in I_R} y_i^2 - \frac{1}{n_R} \left(\sum_{i \in I_R} y_i \right)^2 - \sum_{i \in I} y_i^2 + \frac{1}{n} \left(\sum_{i \in I} y_i \right)^2 \\
& = - \frac{1}{n_L} \left(\sum_{i \in I_L} y_i \right)^2 - \frac{1}{n_R} \left(\sum_{i \in I_R} y_i \right)^2 + \frac{1}{n} \left(\sum_{i \in I} y_i \right)^2 \\
\end{aligned}
$$

In [37]:
def _score(self, sum_y_left, n_left, sum_y_right, n_right, sum_y, n):
    score = -(sum_y_left ** 2) / n_left - (sum_y_right ** 2) / n_right + (sum_y ** 2) / n
    return score

def _find_better_split(self, feature_idx):
    x = self.X.values[self.idxs, feature_idx]
    y = self.y[self.idxs]
    # we sort the samples by the feature values
    # this will help us to pin down the threshold values
    sort_idx = np.argsort(x)
    sort_y, sort_x = y[sort_idx], x[sort_idx]
    sum_y, n = sort_y.sum(), len(sort_y)
    sum_y_right, n_right = sum_y, n
    sum_y_left, n_left = 0, 0
    
    for i in range(0, self.n - self.min_samples_leaf):
        xi, xi_next, yi = sort_x[i], sort_x[i + 1], sort_y[i]
        sum_y_left += yi
        sum_y_right -= yi
        n_left += 1
        n_right -= 1
        if n_left < self.min_samples_leaf or xi == xi_next:
            # continue to search for the next split
            continue
        score = self._score(sum_y_left, n_left, sum_y_right, n_right, sum_y, n)
        
        if score < self.best_score_so_far:
            self.best_score_so_far = score
            self.split_feature_idx = feature_idx
            self.threshold = (xi + xi_next) / 2

In [38]:
DecisionTree._score = _score
DecisionTree._find_better_split = _find_better_split
t = DecisionTree(X, y, min_samples_leaf=5, max_depth=6)
X.columns[t.split_feature_idx], t.threshold

('s5', -0.0037611760063045703)

In [39]:
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [40]:
def __repr__(self):
    s = f'n: {self.n}; value: {self.value:.2f}'
    if not self.is_leaf:
        split_feature_name = self.X.columns[self.split_feature_idx]
        s += f'; split_feature: {split_feature_name} <= {self.threshold:.2f}'
    return s

In [41]:
DecisionTree.__repr__ = __repr__
t = DecisionTree(X, y, min_samples_leaf=5, max_depth=2)
print(t)
print(t.left)
print(t.left.left)

n: 442; value: 152.13; split_feature: s5 <= -0.00
n: 218; value: 109.99; split_feature: bmi <= 0.01
n: 171; value: 96.31


## Prediction

To predict the target variable, we will use the mean of the target variable in the leaf node. And then we could build up the tree recursively.

In [43]:
def predict(self, X):
    return np.array([self._predict_row(xi) for xi in X.values]) 

def _predict_row(self, xi):
    if self.is_leaf:
        return self.value
    t = self.left if xi[self.split_feature_idx] <= self.threshold else self.right
    return t._predict_row(xi)

In [44]:
DecisionTree.predict = predict
DecisionTree._predict_row = _predict_row
t.predict(X.iloc[:3, :])

array([225.87962963,  96.30994152, 225.87962963])

## Full Code

In [50]:
class DecisionTree:
    
    def __init__(self, X: pd.DataFrame, y: pd.Series,
                 min_samples_leaf: int = 5,
                 max_depth: int = 6,
                 idxs: np.ndarray = None):
        assert max_depth >= 0
        assert min_samples_leaf > 0
        self.min_samples_leaf, self.max_depth = min_samples_leaf, max_depth
        if isinstance(y, pd.Series):
            y = y.values
        if idxs is None:
            idxs = np.arange(len(y))
        self.X, self.y, self.idxs = X, y, idxs
        self.n, self.m = len(idxs), X.shape[1]
        # value of the node is the mean of y
        self.value = np.mean(y[idxs])
        self.best_score_so_far = float('inf') # initial loss before split finding
        
        if self.max_depth > 0:
            self._maybe_insert_child_nodes()
            
    @property
    def is_leaf(self):
        return self.best_score_so_far == float('inf')
    
    def _maybe_insert_child_nodes(self):
        for j in range(self.m):
            # there exist a better split for feature j
            # such as value < 50 for color red in our example
            # we will implement this method later
            self._find_better_split(j)
        if self.is_leaf:
            # no need to create child nodes
            return
        x = self.X.values[self.idxs, self.split_feature_idx]
        # get the indexes of the samples that are less than 
        # or equal to the split value
        # this step is to sort the samples to the left and right nodes
        left_idx = np.nonzero(x <= self.threshold)[0]
        right_idx = np.nonzero(x > self.threshold)[0]
        self.left = DecisionTree(self.X, self.y, self.min_samples_leaf, self.max_depth - 1, self.idxs[left_idx])
        self.right = DecisionTree(self.X, self.y, self.min_samples_leaf, self.max_depth - 1, self.idxs[right_idx])
        
    def _score(self, sum_y_left, n_left, sum_y_right, n_right, sum_y, n):
        score = -(sum_y_left ** 2) / n_left - (sum_y_right ** 2) / n_right + (sum_y ** 2) / n
        return score

    def _find_better_split(self, feature_idx):
        x = self.X.values[self.idxs, feature_idx]
        y = self.y[self.idxs]
        # we sort the samples by the feature values
        # this will help us to pin down the threshold values
        sort_idx = np.argsort(x)
        sort_y, sort_x = y[sort_idx], x[sort_idx]
        sum_y, n = sort_y.sum(), len(sort_y)
        sum_y_right, n_right = sum_y, n
        sum_y_left, n_left = 0, 0
        
        for i in range(0, self.n - self.min_samples_leaf):
            xi, xi_next, yi = sort_x[i], sort_x[i + 1], sort_y[i]
            sum_y_left += yi
            sum_y_right -= yi
            n_left += 1
            n_right -= 1
            if n_left < self.min_samples_leaf or xi == xi_next:
                # continue to search for the next split
                continue
            score = self._score(sum_y_left, n_left, sum_y_right, n_right, sum_y, n)
            
            if score < self.best_score_so_far:
                self.best_score_so_far = score
                self.split_feature_idx = feature_idx
                self.threshold = (xi + xi_next) / 2

                
    def __repr__(self):
        s = f'n: {self.n}; value: {self.value:.2f}'
        if not self.is_leaf:
            split_feature_name = self.X.columns[self.split_feature_idx]
            s += f'; split_feature: {split_feature_name} <= {self.threshold:.2f}'
        return s
    
    def predict(self, X):
        return np.array([self._predict_row(xi) for xi in X.values]) 

    def _predict_row(self, xi):
        if self.is_leaf:
            return self.value
        t = self.left if xi[self.split_feature_idx] <= self.threshold else self.right
        return t._predict_row(xi)

In [46]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

X, y = fetch_california_housing(as_frame=True, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=43)

In [51]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

max_depth = 8
min_samples_leaf = 16

tree = DecisionTree(X_train, y_train, max_depth=max_depth, min_samples_leaf=min_samples_leaf)
pred = tree.predict(X_test)

sk_tree = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
sk_tree.fit(X_train, y_train)
sk_pred = sk_tree.predict(X_test)

print(f'from scratch MSE: {mean_squared_error(y_test, pred):0.4f}')
print(f'scikit-learn MSE: {mean_squared_error(y_test, sk_pred):0.4f}')

from scratch MSE: 0.3988
scikit-learn MSE: 0.3988
